In [1]:
import requests
from lxml import html

import redis
from random import randint
import time

from ipywidgets import Image
import plotly.graph_objs as go

In [2]:
from mlrepricer import helper, redisdb, helper
from mlrepricer.oldsql import schemas
from mlrepricer.oldsql.database import SQLite
import pandas as pd
import numpy as np

How you get started with redis.

In [3]:
r = redis.StrictRedis(**helper.rediscred, decode_responses=True)
redisdb.server_start()

Iterate over pictures:

In [4]:
pictures = dict()
for asin in r.scan_iter():
    if asin.startswith('pic'):
        pictures.update({asin: list(r.smembers(asin))[0]})

In [5]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

In [6]:
filter1 = helper.cleanup(df)
norm = helper.normalize(filter1)
norm.head()

,asin,feedback,feedbackpercent,isbuyboxwinner,price,sellerid,time_changed,shipping_time_0,shipping_time_46,shipping_time_48,shipping_time_72
ID,,,,,,,,,,,
1,B074XTQL3D,0.878661,0.9604,1,14.17344,ADBIN7JOKZABU,2018-05-13 10:31:00.953,0,0,0,1
2,B074XTQL3D,0.313549,0.8281,0,14.17344,A6SCPSJ316ST,2018-05-13 10:31:00.953,0,0,1,0
3,B074XTQL3D,0.738088,0.9604,0,14.45691,A2EHWGAW6J9W8Q,2018-05-13 10:31:00.953,0,0,0,1
4,B074XTQL3D,0.363759,0.7921,0,17.52354,A1TY2HQMSLJUR2,2018-05-13 10:31:00.953,0,0,0,1
5,B074XTQL3D,0.523644,0.7744,0,21.41481,A392IUIG1QL0VK,2018-05-13 10:31:00.953,0,0,1,0


In [7]:
fig = go.FigureWidget(
    data=[
        dict(
            x=norm['time_changed'],
            y=norm['price'],
        )
    ],
)

In [8]:
scatter = fig.data[0]
scatter.text = norm['sellerid']
scatter.hoverinfo = 'text'

In [9]:
from ipywidgets import HTML
details = HTML()

In [10]:
def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    details.value = norm.iloc[ind].to_frame().to_html()

scatter.on_hover(hover_fn)

In [17]:
from ipywidgets import Image, Layout
image_widget = Image(
    value=pictures['pic_B075NJNLJT'],
    layout=Layout(height='400px', width='400px')
)
image_widget

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\n\x07\x07\…

In [12]:
def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    
    # Update details HTML widget
    details.value = norm.iloc[ind].to_frame().to_html()
    
    # Update image widget
    model_year = 'pic_' + norm['asin'][ind]
    image_widget.value = m[model_year]

scatter.on_hover(hover_fn)

In [18]:
from ipywidgets import HBox, VBox
VBox([fig,
      HBox([image_widget, details])])

    'data': [{'hoverinfo': 'text',
              'text': array(['ADBIN7JOKZABU',…

In [14]:
# that would be one picture
pic = list(r.smembers('pic_B01F2RLCJ8'))[0]
#r.delete('pic_B01J670I36')
# lets use the list of pictures, you can change the index to play around
Image(value=m[49])

NameError: name 'm' is not defined

In [15]:
def download_image(asin):
    country = 'de'
    site = requests.get(f'https://amazon.{country}/dp/{asin}')
    parser = html.fromstring(site.text)
    IMG = "//img[@id='landingImage']"
    image_link = parser.xpath(IMG)[0].get('src')
    image = requests.get(image_link).content
    assert isinstance(image, bytes)
    return image
    
def dump_into_redis(image):
    r.sadd(f'pic_{asin}', image)

In [16]:
for asin in r.scan_iter():
    if asin != 'updated_asins' and not asin.startswith('pic'):
        if not r.exists('pic_'+asin):
            print(asin)
            time.sleep(randint(2,4))
            try:
                dump_into_redis(download_image(asin))
            except:
                time.sleep(randint(5, 6))

B07BGJ72FC
B07BGBXBMG


KeyboardInterrupt: 